* 전처리에 필요한 모듈 (띄어쓰기)

해당 명령어는 실행 시킨뒤에 다시 런타임 재시도 해야함 ! 


In [ ]:
% cd "/content/drive/My Drive/py-hanspell/"

In [ ]:
! python setup.py install

In [ ]:
! pip install py-hanspell

In [ ]:
! pip install tqdm --upgrade

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm

# 전처리 모듈
import re
from hanspell import spell_checker

# 크롤링 모듈 
from bs4 import BeautifulSoup
import requests

##1. Data Load

In [ ]:
def data_load():
    file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
    train_data = pd.read_csv(file_path+"train_data_pre1_0820.csv")
    test_data = pd.read_csv(file_path+"test_data_pre1_0820.csv")
    return train_data, test_data

In [ ]:
train_data , test_data = data_load()
train_p_setlist = list(set(train_data['p_code']))
print(len(train_p_setlist)) # len 2124
chk_test = list(set(test_data.loc[(test_data['p_code'].isin(train_p_setlist))]['p_code']))
print(len(chk_test)) # len 27

## 1.5 data check

### 네이버 상품 품목과 상품 명 합쳐서 검색하기 (검색 결과 더 좋음) 
* 상품 품목 + 이름 합쳐서 검색 시도 
* 이때 상품 품목 NS 상품코드 --> 네이버 상품코드로 바꿔주기 
* p_code —> 네이버 대분류 코드로 맵핑 --> " 네이버 상품 코드 + 상품명 " 으로 검색 시도 
* ex. 엘린느 바지 -> 패션의류 엘린느 바지 


In [ ]:
naver_cate_list = ['가구', '가전', '건강식품', '식품', '무형', '생활', '속옷', '패션의류', '미용', '패션잡화' , '주방용품', '침구']
p_cate_list = sorted(list(set(train_data['p_cate'])))
print(p_cate_list)
pcate_2navercate_dict = dict(zip(p_cate_list, naver_cate_list))

['가구', '가전', '건강기능', '농수축', '무형', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구']


In [ ]:
def pcate_2navercate(p_cate):
    return pcate_2navercate_dict[p_cate]

In [ ]:
train_data['naver_cate'] = train_data['p_cate'].apply(pcate_2navercate)
train_data['naver_cate_name'] = train_data['naver_cate'] +" "+train_data['p_name']
display(train_data.head(10))

test_data['naver_cate'] = test_data['p_cate'].apply(pcate_2navercate)
test_data['naver_cate_name'] = test_data['naver_cate'] +" "+test_data['p_name']
display(test_data.head(10))

In [ ]:
# ## test data 와 traindata 겹치는 상품코드만 거르기  --> 체크 완료 !! 

# result_list = []
# for chk_p_code in list(chk_test):
#     print(chk_p_code)
#     result1, result2 = False, False
#     train_checker = set(train_data[train_data['p_code'] == chk_p_code]['p_name'])
#     test_checker = set(test_data[test_data['p_code'] == chk_p_code]['p_name'])

#     if len(train_checker) == 1 and len(train_checker) == 1:
#         result1 = True 
#     if train_checker == test_checker:
#         # print(train_checker, test_checker)
#         result2 = True

#     result_list.append(result1)
#     result_list.append(result2)

# if False in result_list:
#     print("ERROR")
# else: 
#     print("P_code and P_name are same in each dataframe")

In [ ]:
### 무형 DF - concat train/ test
em_train_df = train_data[train_data['p_cate'] == '무형']
print(em_train_df.shape)
em_test_df = test_data[test_data['p_cate'] == '무형']
print(em_test_df.shape)
em_concat_df = pd.concat([em_train_df, em_test_df])
print(em_concat_df.shape)

(937, 17)
(175, 16)
(1112, 17)


In [ ]:
## 무형 train/ test 안의 상품 코드들 다 빼두기 
train_check = len(set(train_data[train_data['p_cate'] == '무형']['p_code'].to_list()))
test_check = len(set(test_data[test_data['p_cate'] == '무형']['p_code'].to_list()))
print(train_check+test_check)
em_pcode_list = list(set(em_concat_df['p_code'].to_list())) ## 무형중 겹치는거 3개 빠짐
print(len(em_pcode_list))

110
107


## 2. train. test p_code concat --> make dictionary and DF

In [ ]:
# make dictionary code to product name 

code2name_dict_train = dict(zip(train_data['p_code'].to_list(), train_data['naver_cate_name'].to_list()))
print(len(code2name_dict_train))
code2name_dict_test = dict(zip(test_data['p_code'].to_list(), test_data['naver_cate_name'].to_list()))
print(len(code2name_dict_test))
code2name_dict_train.update(code2name_dict_test)
code2name_dict = code2name_dict_train
print(len(code2name_dict))

2124
417
2514


In [ ]:
# make data frame - unique p_code and p_name
code2name_df = pd.DataFrame.from_dict(code2name_dict.items())
code2name_df.columns = ['p_ocde', 'naver_cate_name']
display(code2name_df)

## SAVE DF 
file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
code2name_df.to_csv(file_path+"crawl_output_0904/unique_pcode_traintest_df(navercate_prep).csv",index = False)

,p_ocde,naver_cate_name
0,201072,패션의류 테이트 남성 셀린니트3종
1,201079,패션의류 테이트 여성 셀린니트3종
2,200974,속옷 오모떼 레이스 파운데이션 브라
3,202377,패션의류 CERINI by PAT 남성 소프트 기모 릴렉스팬츠
4,202404,패션의류 보코 리버시블 무스탕
...,...,...
2509,202320,패션잡화 [루이띠에] 24K 999 순금 11.25g 샤인 볼륨 반지
2510,200862,식품 (특대사이즈) 안동간고등어 13팩
2511,200019,식품 AAC녹두삼계탕8팩
2512,200022,식품 AAC녹두삼계탕죽세트


## 3. Crawling

### 3.1 Crawl codes

In [ ]:
def crawl(keyword):
    url = f"https://search.shopping.naver.com/search/all.nhn?query={keyword}&cat_id=&frm=NVSHATC"
    data = requests.get(url)
    return data.content

In [ ]:
def getCrawlInfo(li):
    # get Productname, link, price 
    aTit = li.find("a", {"class": "basicList_link__1MaTN"})
    name = aTit.text
    link = aTit['href']
    price = li.find("span", {"class":"price_num__2WUXn"}).text.replace(",", "")

    # get Categories - 대/ 중/ 소  (없으면 None 으로 표기되도록)
    cate_class = li.findAll("a", {"class": "basicList_category__wVevj"})
    # print(cate_class)
    cate_list = [None, None, None, None]  # 0904 수정 - 카테고리 4개로 확장 
    for idx in range(len(cate_class)): 
        if cate_class[idx] :
            cate_list[idx] = cate_class[idx].text

    result_dict = {"name":name, "link": link, "price":price, "cate1":cate_list[0] , "cate2":cate_list[1], "cate3":cate_list[2], "cate4":cate_list[3]}
    # print("RESULT DICT", result_dict)
    return result_dict

In [ ]:
def parse(pageString):
    bsObj = BeautifulSoup(pageString, "html.parser")
    ul = bsObj.find("ul", {"class":"list_basis"})

    if ul: # 검색 결과 있는 경우 
        lis = ul.find("li",{"class":"basicList_item__2XT81"}) 
        crawl_info_dict = getCrawlInfo(lis)
        crawl_info_dict['cr'] = 1 # 검색 결과 여부도 labeling - (결과 있으면 1)

    else: # 검색 결과 없는 경우 
        key_list = ["name", "link", "price", "cate1", "cate2", "cate3", "cate4"]
        crawl_info_dict = dict(zip(key_list, [None]*6))
        crawl_info_dict['cr'] = 0  # 검색 결과 여부도 labeling - (결과 없으면 0)

    return crawl_info_dict

In [ ]:
def get_crawl_data(p_code, p_name):
    try : 
        pageString = crawl(p_name)
        crawl_info_dict = parse(pageString)
        crawl_info_dict['p_code'] = p_code
    except: 
        key_list = ["name", "link", "price", "cate1", "cate2", "cate3", "cate4"]
        crawl_info_dict = dict(zip(key_list, [None]*len(key_list)))
        crawl_info_dict['cr'] = -1  #에러 결과 여부도 labeling -1
        crawl_info_dict['p_code'] = p_code
    return crawl_info_dict

### 3.2 preprocessing again (크롤 안되었을경우 전처리하고 재검색 되도록)

In [ ]:
def prep1(text):
    
    text = re.sub('무이자', '', text)
    text = re.sub('일시불', '', text)
    text = re.sub(r'[^a-zA-Zㄱ-힗\s]+[a-zA-Zㄱ-힗]+[^a-zA-Zㄱ-힗\s]', ' ',text)
    # print(text)
    # text = re.sub('패키지', '', text)
    # text = re.sub('세트', '', text)
    text = re.sub('예약', '', text)
    text = re.sub('상담', '', text)
    text = re.sub('(무)', '', text)
    text = re.sub('(일)', '', text)
    
    text = re.sub(r'[0-9]+인용', '', text)
    text = re.sub(r'[0-9]+팩', '', text)
    text = re.sub(r'[0-9]+종', '', text)

    text = re.sub(r'[^a-zA-Zㄱ-힗]', ' ',text)
    text = text.split()
    # text = [tok for tok in text if l[^a-zA-Zㄱ-힗]en(tok)>1]
    text = " ".join(text)

    # 띄어쓰기
    result = spell_checker.check(text)
    text = ' '.join([key for key in result.words.keys() if len(key)>1])
    text = re.sub(r'\<[^)]*\>', '', text)

    return text

In [ ]:
sample_text = '주방용품 (도냄)무이자 쿠쿠전기밥솥 10인용'
print(prep1(sample_text))

주방용품 쿠쿠 전기밥솥


### 3.3 main

In [ ]:
def main(code2name_dict):
    all_dict_list = []
    for p_code, p_name in tqdm(code2name_dict.items()):
        # print()
        if p_name !=0:
            one_crawl_dict = get_crawl_data(p_code, p_name)
            if one_crawl_dict['cr'] != 1: # 검색이 되지 않은 경우 : 전체 단어 수에서 2개만 남을때까지 뒤에서 줄여나가며 재검색    

                # 1. 전처리 시도 
                up_p_name = prep1(p_name) 
                # print("PREP",p_name,"-->",up_p_name)
                one_crawl_dict = get_crawl_data(p_code, up_p_name)


                # 전처리 해도 검색 되지 않는 경우 
                if one_crawl_dict['cr'] != 1 :
                    pop_up_p_name = up_p_name+'' 
                    p_name_list = pop_up_p_name.split()
                    total_len = len(p_name_list)


                    # 2. 글자수 뒤에서 부터 하나씩 줄여서 3개만 빼보기
                    if total_len > 3: 
                        for _ in range(3):
                            # print("update name -->", pop_up_p_name)
                            p_name_list.pop() # 맨뒤의 글자 하나 빼기 
                            pop_up_p_name = ' '.join(p_name_list) # 다시 문자열로 변환해서 
                            one_crawl_dict = get_crawl_data(p_code, pop_up_p_name)
                            if one_crawl_dict['name']:
                                break 
                            else:
                                p_name_list = pop_up_p_name.split()   
                    else: # 단어가 4개 토큰 그냥 pass
                        pass

            if one_crawl_dict['cr'] != 1: print(p_name) # 끝까지 검색 안된 상품명 출력 
            all_dict_list.append(one_crawl_dict)
            # print(p_code, p_name, one_crawl_dict)
    return all_dict_list


In [ ]:
## CRAWLING ! 
all_dict_list = main(code2name_dict)

  4%|▎         | 90/2514 [03:47<2:24:18,  3.57s/it]

미용 마리끌레르 파리컬렉션 리얼 하이드로 립스틱(리뉴얼)


  4%|▎         | 91/2514 [03:53<2:55:13,  4.34s/it]

패션의류 아리스토우 남성기모티셔츠5종


  4%|▍         | 108/2514 [04:46<2:33:49,  3.84s/it]

...

 96%|█████████▌| 2401/2514 [1:56:48<08:13,  4.36s/it]

생활 에브리빙 기능성 쿨잠패드 세트 SS(슈퍼싱글)


 96%|█████████▌| 2403/2514 [1:56:57<08:34,  4.64s/it]

생활 파격할인 프리미엄 니봇 시그니처 로봇청소기 + 셰퍼데일리 청소기


100%|██████████| 2514/2514 [2:01:37<00:00,  2.90s/it]


In [ ]:
import json
file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
result_file = open(file_path+"crawl_output_0904/result_dict(0904).json",'w+')
result_file.write(json.dumps(all_dict_list))
crawling_df = pd.read_json(file_path+"crawl_output_0904/result_dict(0904).json")
crawling_df = crawling_df[['p_code','cr','name','link','price','cate1','cate2','cate3', "cate4"]]
crawling_df.to_csv(file_path+"crawl_output_0904/crawl_uniquecode(0904).csv", index = False)

3168144

In [ ]:
crawling_df[crawling_df['cr'] !=1]

,p_code,cr,name,link,price,cate1,cate2,cate3,cate4
89,200881,0,None,None,None,None,None,None,None
90,201059,0,None,None,None,None,None,None,None
107,202076,0,None,None,None,None,None,None,None
115,201058,0,None,None,None,None,None,None,None
116,201061,0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
2299,200380,0,None,None,None,None,None,None,None
2378,201809,0,None,None,None,None,None,None,None
2397,200323,0,None,None,None,None,None,None,None
2400,200324,0,None,None,None,None,None,None,None


In [ ]:
### result check 
chk_result = pd.read_csv(file_path+"crawl_output_0904/crawl_uniquecode(0904).csv")
print("CRAWL FAILED", len(chk_result[chk_result['cr']!=1]))

CRAWL FAILED 95


## 3.5 크롤 결과중 무형은 빼기 !!!!!!

In [ ]:
# 무형이 아닌 상품코드 리스트 
not_emp_data = train_data[train_data['p_cate']!='무형']
not_emp_pcode = not_emp_data['p_code'].to_list()
not_emp_pcode_list = list(set(not_emp_pcode))
not_emp_crawling_df = crawling_df.loc[(crawling_df['p_code'].isin(not_emp_pcode_list))]
not_emp_crawling_df = not_emp_crawling_df.reset_index(drop = True)
not_emp_crawling_df

,p_code,cr,name,link,price,cate1,cate2,cate3,cate4
0,201072,1,테이트 남여공용 베이직 터틀넥 3종 KA8W9-MSP010 MSP020 USP010,https://cr.shopping.naver.com/adcr.nhn?x=7pGpB...,7310원,패션의류,남성의류,니트/스웨터,None
1,201079,1,테이트 여성 19FW 헤어리 폭스블렌디드 니트 3종,https://cr.shopping.naver.com/adcr.nhn?x=3ZVe2...,37110원,패션의류,여성의류,니트/스웨터,None
2,200974,1,[BH0428] 신품 삼각컵 브라 쁘띠파운데이션삼중지내의 여성감 소녀레이스 노와이어 얇은,https://cr.shopping.naver.com/adcr.nhn?x=mKZYN...,26260원,패션의류,여성언더웨어/잠옷,브라,None
3,202377,1,"CERINI by PAT 소프트 기모 릴렉스 팬츠 3종 (블랙,네이비,그레이)",https://cr.shopping.naver.com/adcr.nhn?x=6R5nd...,58900원,패션의류,남성의류,바지,None
4,202404,1,더아이잗 리버시블 롱 무스탕 ik9a3fs04,https://cr.shopping.naver.com/adcr.nhn?x=IoJ21...,64860원,패션의류,여성의류,코트,None
...,...,...,...,...,...,...,...,...,...
2037,201357,1,루시헨느 쉐이퍼팬티 9종 홀리데이 패키지 보정속옷 여성솟옷,https://cr.shopping.naver.com/adcr.nhn?x=siCcd...,9590원,패션의류,여성언더웨어/잠옷,팬티,None
2038,201416,1,크리스티나앤코 리버시블 패딩코트,https://cr.shopping.naver.com/adcr.nhn?x=V9nt9...,29900원,패션의류,여성의류,점퍼,None
2039,201819,1,레스토닉 블루투스 LED 서랍 침대 슈퍼싱글 퀸 매트포함,https://cr.shopping.naver.com/adcr.nhn?x=fodRm...,470810원,가구/인테리어,침실가구,침대,침대프레임
2040,201821,1,레스토닉 블루투스 LED 서랍 침대 슈퍼싱글 퀸 매트포함,https://cr.shopping.naver.com/adcr.nhn?x=bSsh5...,470810원,가구/인테리어,침실가구,침대,침대프레임


## 4. 크롤링 안된 상품명들 확인 

In [ ]:
def code2_navercate_name(p_code):
    return code2name_dict[p_code]

In [ ]:
# get p name of not cr data 
not_cr_data = not_emp_crawling_df[not_emp_crawling_df['cr'] != 1]
not_cr_data['p_name'] = not_cr_data['p_code'].apply(code2_navercate_name)
not_cr_data = not_cr_data[['p_code','p_name']]
not_cr_data.to_csv(file_path+"crawl_output_0904/crawl_uniquecode_failed(0904).csv", index = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


**** 과거 크롤링 데이터에서 해당 부분 메워진거 있는지 확인해보기 

In [ ]:
def before_cr_data_load():
    # chk_result = pd.read_csv(file_path+"crawl_output_0904/crawl_uniquecode(0904).csv")
    file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
    not_cr_data = pd.read_csv(file_path+"crawl_output_0904/crawl_uniquecode_failed(0904).csv")
    my_cr_data = pd.read_csv(file_path+"crawl_output_0904/before_cr_output/navercrawldata_ver1(0826).csv") # 내 크롤데이터
    edit_cr_data = pd.read_csv(file_path+"crawl_output_0904/before_cr_output/categories_train.csv") # 아이템 스카우트 크롤데이터
    edit_cr_data.columns = ['p_code', 'p_name', 'cate2', 'cate3']
    return not_cr_data, my_cr_data, edit_cr_data 

In [ ]:
not_cr_data, my_cr_data, edit_cr_data  = before_cr_data_load()

In [ ]:
temp_merge1 = pd.merge(not_cr_data, my_cr_data, left_on = 'p_code', right_on = 'p_code', how = 'left')
temp_merge1['cr'] = temp_merge1.cr.fillna(0)
temp_merge1.to_csv(file_path+"crawl_output_0904/crawl_uniquecode_mergefailed(0904).csv", index = False)
print(len(temp_merge1[temp_merge1['cr'] ==1])) # 57개 가져올 수 있음 
print(temp_merge1['cr'].value_counts())

57
1    57
0    17
Name: cr, dtype: int64


## 5. 노가다 한거 들고 오기

In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
finaledit_cr_data = pd.read_csv(file_path+"crawl_output_0904/crawl_uniquecode_mergefailed_edit(0905).csv")
finaledit_dict = dict(zip(finaledit_cr_data['p_code'].to_list(),finaledit_cr_data['search_word'].to_list()))

In [ ]:
finaledit_cr_list = main(finaledit_dict)

100%|██████████| 95/95 [03:01<00:00,  1.91s/it]


In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
result_file = open(file_path+"crawl_output_0904/result_final_editdict(0905).json",'w+')
result_file.write(json.dumps(finaledit_cr_list))
finaledit_cr_df = pd.read_json(file_path+"crawl_output_0904/result_final_editdict(0905).json")
finaledit_cr_df = finaledit_cr_df[['p_code','cr','name','link','price','cate1','cate2','cate3', "cate4"]]
finaledit_cr_df.to_csv(file_path+"crawl_output_0904/crawl_uniquecode_editfinal(0905).csv", index = False)

In [ ]:
### 아까 크롤 잘된거랑 마지막 노가다 작업이랑 합치기 
print(f"전체 {len(crawling_df)} : \n{len(crawling_df[crawling_df['cr']==1])} : crawled /\n{len(crawling_df[crawling_df['cr']!=1])} : not crawled ")
print("not crawled 중 노가다 개수", len(finaledit_cr_df))

전체 2514 : 
2419 : crawled /
95 : not crawled 
not crawled 중 노가다 개수 95


## 6. 전체 통합 - 무형 상품 코드에 해당되는 행 제거 

In [ ]:
concat_all_cr_data = pd.concat([crawling_df[crawling_df['cr']==1],finaledit_cr_df])
concat_all_cr_data = concat_all_cr_data.loc[concat_all_cr_data['p_code'].isin(em_pcode_list) == False]
concat_all_cr_data = concat_all_cr_data.reset_index(drop = True)
concat_all_cr_data.to_csv(file_path+"crawl_output_0904/Pcode_crawl_all.csv", index = False)

In [ ]:
concat_all_cr_data.loc[concat_all_cr_data['p_code'].isin(em_pcode_list)] ## 무형에 속한 데이터 다 제거된 것을 확인 ! 

,p_code,cr,name,link,price,cate1,cate2,cate3,cate4


In [ ]:

concat_all_cr_data = pd.read_csv(file_path+"crawl_output_0904/Pcode_crawl_all.csv")

In [ ]:
concat_all_cr_data

,p_code,cr,name,link,price,cate1,cate2,cate3,cate4
0,201072,1,테이트 남여공용 베이직 터틀넥 3종 KA8W9-MSP010 MSP020 USP010,https://cr.shopping.naver.com/adcr.nhn?x=7pGpB...,7310원,패션의류,남성의류,니트/스웨터,NaN
1,201079,1,테이트 여성 19FW 헤어리 폭스블렌디드 니트 3종,https://cr.shopping.naver.com/adcr.nhn?x=3ZVe2...,37110원,패션의류,여성의류,니트/스웨터,NaN
2,200974,1,[BH0428] 신품 삼각컵 브라 쁘띠파운데이션삼중지내의 여성감 소녀레이스 노와이어 얇은,https://cr.shopping.naver.com/adcr.nhn?x=mKZYN...,26260원,패션의류,여성언더웨어/잠옷,브라,NaN
3,202377,1,"CERINI by PAT 소프트 기모 릴렉스 팬츠 3종 (블랙,네이비,그레이)",https://cr.shopping.naver.com/adcr.nhn?x=6R5nd...,58900원,패션의류,남성의류,바지,NaN
4,202404,1,더아이잗 리버시블 롱 무스탕 ik9a3fs04,https://cr.shopping.naver.com/adcr.nhn?x=IoJ21...,64860원,패션의류,여성의류,코트,NaN
...,...,...,...,...,...,...,...,...,...
2402,200380,1,보몽드 [보몽드]시어서커 실루엣 차렵이불(SS/Q),https://cr.shopping.naver.com/adcr.nhn?x=Jn0Kd...,27500원,가구/인테리어,침구단품,차렵이불,NaN
2403,201809,1,신일전자 신일 에어쿨러 엣지 플러스 냉풍기 SIF-P75MD 가정용,https://cr.shopping.naver.com/adcr.nhn?x=cRJmC...,160000원,디지털/가전,계절가전,냉풍기,일반용냉풍기
2404,200323,1,Shop 에브리빙 기능성 쿨잠패드 세트 슈퍼싱글,https://cr.shopping.naver.com/adcr.nhn?x=tqaPu...,66670원,가구/인테리어,침구세트,이불패드세트,더블/퀸이불패드세트
2405,200324,1,Shop 에브리빙 기능성 쿨잠패드 세트 슈퍼싱글,https://cr.shopping.naver.com/adcr.nhn?x=yrk5v...,66670원,가구/인테리어,침구세트,이불패드세트,더블/퀸이불패드세트


** 성별 매칭 안되는 제품있음, 

In [ ]:
def chk_name_cate2(one):
    name = one['name']
    cate2 = one['cate2']
    if '여성' in name:
        if cate2 =='남성신발':
            return "여성신발"
        elif cate2 =='남성의류':
            return "여성의류"
        else:
            return cate2
    
    elif '남성' in name:
        if cate2 =='여성신발':
            return "남성신발"
        elif cate2 =='여성의류':
            return "남성의류"
        else:
            return cate2
    else:
        return cate2

In [ ]:
concat_all_cr_data['cate2'] = concat_all_cr_data[['name','cate2']].apply(chk_name_cate2, axis=1)

In [ ]:
concat_all_cr_data.to_csv(file_path+"crawl_output_0904/Pcode_crawl_all_0905.csv", index = False)

In [ ]:
concat_all_cr_data

,p_code,cr,name,link,price,cate1,cate2,cate3,cate4
0,201072,1,테이트 남여공용 베이직 터틀넥 3종 KA8W9-MSP010 MSP020 USP010,https://cr.shopping.naver.com/adcr.nhn?x=7pGpB...,7310원,패션의류,남성의류,니트/스웨터,NaN
1,201079,1,테이트 여성 19FW 헤어리 폭스블렌디드 니트 3종,https://cr.shopping.naver.com/adcr.nhn?x=3ZVe2...,37110원,패션의류,여성의류,니트/스웨터,NaN
2,200974,1,[BH0428] 신품 삼각컵 브라 쁘띠파운데이션삼중지내의 여성감 소녀레이스 노와이어 얇은,https://cr.shopping.naver.com/adcr.nhn?x=mKZYN...,26260원,패션의류,여성언더웨어/잠옷,브라,NaN
3,202377,1,"CERINI by PAT 소프트 기모 릴렉스 팬츠 3종 (블랙,네이비,그레이)",https://cr.shopping.naver.com/adcr.nhn?x=6R5nd...,58900원,패션의류,남성의류,바지,NaN
4,202404,1,더아이잗 리버시블 롱 무스탕 ik9a3fs04,https://cr.shopping.naver.com/adcr.nhn?x=IoJ21...,64860원,패션의류,여성의류,코트,NaN
...,...,...,...,...,...,...,...,...,...
2402,200380,1,보몽드 [보몽드]시어서커 실루엣 차렵이불(SS/Q),https://cr.shopping.naver.com/adcr.nhn?x=Jn0Kd...,27500원,가구/인테리어,침구단품,차렵이불,NaN
2403,201809,1,신일전자 신일 에어쿨러 엣지 플러스 냉풍기 SIF-P75MD 가정용,https://cr.shopping.naver.com/adcr.nhn?x=cRJmC...,160000원,디지털/가전,계절가전,냉풍기,일반용냉풍기
2404,200323,1,Shop 에브리빙 기능성 쿨잠패드 세트 슈퍼싱글,https://cr.shopping.naver.com/adcr.nhn?x=tqaPu...,66670원,가구/인테리어,침구세트,이불패드세트,더블/퀸이불패드세트
2405,200324,1,Shop 에브리빙 기능성 쿨잠패드 세트 슈퍼싱글,https://cr.shopping.naver.com/adcr.nhn?x=yrk5v...,66670원,가구/인테리어,침구세트,이불패드세트,더블/퀸이불패드세트
